In [ ]:
# 4 Tiny helper functions

def print_word_list(wl):
    print wl
    str =  ' '.join(wl)
    pattern = re.compile('\s\W\s')
    shift = 0
    for ma in pattern.finditer(str):
        str = str[:ma.start(0)-shift]+ma.group(0)[1:]+ str[ma.end(0)-shift:]
        shift +=1
    if str[-2] == ' ':
        str = str[:-2] + str[-1:]
    return str

def get_print(sentence_or_similar):
    if type(sentence_or_similar) is gensim.models.doc2vec.LabeledSentence:
        word_list = sentence_or_similar.words
    elif type(sentence_or_similar) is int64 or type(sentence_or_similar) is int: # just an index
        word_list = sentences[sentence_or_similar].words
    else: # TaggedDocument class
        word_list = sentences[int(sentence_or_similar[0][5:])][0]
    return print_word_list(word_list)
    
def get_index_tag(sentence):
    return sentence.tags[0]

def get_index(sentence_or_similar):
    if type(sentence_or_similar) is gensim.models.doc2vec.LabeledSentence:
        return int64(get_index_tag(sentence_or_similar)[5:])
    else:
        return int64(sentence_or_similar[0][5:])
    
def equal_word_lists(index1, index2):
    wl1 = sentences[index1].words
    wl2 = sentences[index2].words
    if len(wl1) != len(wl2):
        return False
    else:
        for i in range(len(wl1)):
            if wl1[i] != wl2[i]:
                return False
    return True

def get_lab_sent_by_similar(similar):
    print get_index(similar)
    return sentences[get_index(similar)]

def get_similarity_by_index(index1, index2):
    return model.docvecs.similarity(index1,index2)

# HTML Helper
def pack_into_elem(tag, clazz, content):
    return '<' + tag + ' class="' + clazz + '"> ' + content+ ' </' + tag +'>'

pre = '''<style>
          .act {font-weight: bold}
          .i {color: grey}
          .sim {color: orange}
          .n {color: blue}
          .p {color: red}
          .r {color: green}          
     </style>'''


# 7 Story Treenode class

PARENT = -1
QUIT = -2
NEXT = -3

num_similars = 20
num_random = 20

class LabSentTreeNode:
    
    def __init__(self, labeledSentence, parent = None):
        self.sentence = labeledSentence
        self.sentence_index = get_index(self.sentence)
        self.similars = self.get_similars()  
        self.randoms = self.get_randoms()
        self.children = {} # index: SentenceTreeNode
        self.selected_child = '' # None
        self.parent = parent
             
    def get_similars(self):
        return model.docvecs.most_similar(get_index_tag(self.sentence),topn = num_similars)
    
    def get_randoms(self):
        randoms = []
        for index in range(num_random):
            rnd_sen = sentences[random.randint(0,len(sentences))]
            randoms.append(rnd_sen)     
        return randoms
                
    def print_options(self):
        for index, sentence in enumerate(self.similars):
            add = '(*)' if get_similar_index(sentence) in self.children else ''
            print index, add, get_print(sentence), "%.3f" % sentence[1]
        if self.parent:
            print 'p: ', get_print(self.parent.sentence)
        if self.sentence_index < len(sentences) - 2:
            print 'n: ', get_print(sentences[self.sentence_index + 1])           
        for index,sentence in enumerate(self.randoms):
            print 'r'+str(index) +": ",  get_print(sentence)
            
    def get_options_html(self):
        html = ''
        for index, sentence in enumerate(self.similars):
            content = pack_into_elem('span','',get_print(sentence))
            index = pack_into_elem('span','',str(index)+': ')
            sentence_index = get_index(sentence)
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similar = pack_into_elem('span','sim',("%.3f" % sentence[1]) + ' / ' + str(index_distance) + ' / ' + str(self.sentence_index))
            html += pack_into_elem('div', '', index + content + similar)
        if self.parent:
            content = pack_into_elem('span','',get_print(self.parent.sentence))
            index = pack_into_elem('span','','P: ')
            sentence_index = self.parent.sentence_index
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, self.parent.sentence_index)
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance) + ' / ' + str(self.sentence_index))
            html += pack_into_elem('div', 'p', index + content + similar)
        if self.sentence_index < len(sentences) - 2:
            content = pack_into_elem('span','n',get_print(sentences[self.sentence_index + 1]))
            index = pack_into_elem('span','','N: ')
            sentence_index = self.sentence_index + 1
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, self.sentence_index + 1)
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance) + ' / ' + str(self.sentence_index))
            html += pack_into_elem('div', 'n', index + content + similar)
        for index,sentence in enumerate(self.randoms):
            content = pack_into_elem('span','',get_print(sentence))
            index = pack_into_elem('span','','R'+str(index)+': ')
            sentence_index = get_index(sentence)
            index += '❗️' if sentence_index in added_sentences else ''
            index_distance =  locale.format('%d', abs(self.sentence_index - sentence_index), 1)
            similarity = get_similarity_by_index(self.sentence_index, get_index(sentence))
            similar = pack_into_elem('span','sim',("%.3f" % similarity) + ' / ' + str(index_distance) + ' / ' + str(self.sentence_index))
            html += pack_into_elem('div', 'r', index + content + similar)
        html += pack_into_elem('div', '', 'Q: Quit 💣')
        return html
    
    def get_sentence_html(self):
        return pack_into_elem('p', 'act', get_print(self.sentence))
    
    def select_child(self):
        u_input = raw_input('Next child: ')
        if u_input == 'p':
            selected_index = PARENT
        elif u_input == 'q':
            return None
        elif u_input == 'n':
            selected_index = NEXT
        elif u_input.startswith('r'):
            selected_index = 100 + int(u_input[1:])
        else:
            try:
                selected_index = int(u_input)
            except ValueError:
                return self
        if selected_index >= 0 and selected_index < len(self.similars):
            lab_sent = get_lab_sent_by_similar(self.similars[selected_index])
            child =  LabSentTreeNode(lab_sent, self)
            self.children[u_input] = child
            self.selected_child = u_input
            return child
        elif selected_index >= 100 and selected_index < len(self.randoms) + 100:
            #print 'random sen'
            child =  LabSentTreeNode(self.randoms[selected_index - 100], self)
            self.children[u_input] = child
            self.selected_child = u_input   
            return child
        elif selected_index == PARENT and self.parent:
            return self.parent
        elif selected_index == NEXT:
            child =  LabSentTreeNode(sentences[self.sentence_index + 1], self)
            self.children[u_input] = child
            self.selected_child = u_input   
            return child
        # a weird number
        return self
        
    def toJSON(self):
        children_toJSON = {}
        for child_index in self.children:
            children_toJSON[child_index] = self.children[child_index].toJSON()
            
        return {'sentence':get_print(self.sentence),
                'index':get_index(self.sentence),
               'children':children_toJSON,
                'selected_child':self.selected_child
               }  
    
# 8 Story creator/log helper functions

def get_story(root_node):
    act_sentence = root_node
    story = []
    while act_sentence:
        story.append(get_print(act_sentence.sentence))
        if act_sentence.selected_child  != '':
            act_sentence = act_sentence.children[act_sentence.selected_child]
        else:
            break
    return story

def log_json(root_node):
    with open('logs/log-'+str(number_logs)+'.json','w') as output:
        output.write(json.dumps(root_node.toJSON(),indent=2))
    
def log_story(root_node):
    story = get_story(root_node)
    with open('logs/story-'+str(number_logs)+'.txt','w') as output:
        for l in story:
            output.write(l+'\n')   
    
def print_story(root_node):
    story = get_story(root_node)
    for l in story:
        print(l)   

def dump_story(root_node):
    with open('logs/story-'+str(number_logs)+'.dump','w') as dump_file:
        pickle.dump(root_node,dump_file)
        
# one list comprehension!!!
def concordance(search_word,sen_range = 0):
    sentence_indices = []
    for index,sen in enumerate(sentences):
        wl = sen[0]
        for word in wl:
            if word == search_word:
                sentence_indices.append(index)
#                 print 'o',get_print(index)
                for r in range(sen_range):
                    # prevent missing element
                    sentence_indices.append(index-r)
                    sentence_indices.append(index+r)
    return sentence_indices

def concordance_result(sentences,sen_range = 0):
    if len(sentences) < 10:
        for sentence in sentences:
            print sentence, get_print(sentence)
    else:
        print len(sentences),'sentences. 5 random ones:'
        for i in range(5):
            print sentence, get_print(random.choice(sentences))

def ask_word_input():
    indices = []
    while len(indices) == 0:
        first_input = raw_input('First input: ')
        indices = concordance(first_input)
    print len(indices), 'Sentences with',first_input
    sentence = sentences[random.choice(indices)]
    return sentence
